In [113]:
%load_ext autoreload
%autoreload 2

from torch import Tensor
from scipy.spatial.transform import Rotation
import torch

from scripts.view_animation_usd import apply_transform_homogeneous, assemble_transform_srt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:

# translation rotation and scale
t = Tensor([1, 2, 3])
rotation = Rotation.from_euler('x', 10, degrees=True)
r = Tensor(rotation.as_matrix()).T
s = 3 * torch.ones(3)

# assemble transform matrix
tf = assemble_transform_srt(t, s, r)

N = 10
points = torch.randn(N, 3)

# transform with matrix
points_transformed_homog = apply_transform_homogeneous(points, tf)

# transform manually
points_transformed_gt = points.clone()
points_transformed_gt = points_transformed_gt * s
points_transformed_gt = points_transformed_gt @ r.T
points_transformed_gt = points_transformed_gt + t

assert torch.allclose(points_transformed_homog, points_transformed_gt)

In [116]:
from scripts.view_animation_usd import decompose_transform_srt

rotation = Rotation.from_euler("x", 25, degrees=True)

# translation rotation and scale
t = Tensor([0, 0, 0])
r = Tensor(rotation.as_matrix()).T
s = 3 * torch.ones(3)

# assemble transform matrix
tf = assemble_transform_srt(t, s, r)

# decompose transform matrix
t_, s_, r_ = decompose_transform_srt(tf)

# check
assert torch.allclose(t, t_)
assert torch.allclose(s, s_)
assert torch.allclose(r, r_)

In [156]:
from pytorch3d.renderer import FoVPerspectiveCameras

# Construct a pytorch3d camera from c2w translation and rotation

t = Tensor([0, 0, -3])
rot = Tensor(Rotation.from_euler("y", 20, degrees=True).as_matrix())

# construct c2w
c2w = assemble_transform_srt(t, torch.ones(3), r)

# get w2c
w2c = c2w.inverse()
t_w2c, _, r_w2c = decompose_transform_srt(w2c)

# to construct camera pass transposed r_w2c and t_w2c
cam = FoVPerspectiveCameras(R=r_w2c.T.unsqueeze(0), T=t_w2c.unsqueeze(0))

# recover w2c
w2c_ = cam.get_world_to_view_transform().get_matrix()[0].T
c2w_ = w2c_.inverse()

# recover t and r
t_, _, r_ = decompose_transform_srt(c2w_)

atol = 0.00001
assert torch.allclose(w2c_, w2c, atol=atol)
assert torch.allclose(c2w_, c2w, atol=atol)
assert torch.allclose(t_, t, atol=atol)
assert torch.allclose(r_, r, atol=atol)